# Aurora Bay FAQ Chatbot with Vector Search & Gemini

This notebook demonstrates how to build an intelligent FAQ chatbot using:
- **BigQuery** to store and query FAQ data
- **Vertex AI Embeddings** to generate semantic vector representations
- **BigQuery Vector Search** to find the most relevant FAQ entries
- **Gemini** to generate natural language answers

### Architecture Overview
```
User Question
     ↓
Generate Embedding (text-embedding-005)
     ↓
Vector Search in BigQuery
     ↓
Top-K Relevant FAQs
     ↓
Gemini (gemini-2.0-flash) → Natural Language Answer
```

### Prerequisites
- A Google Cloud Project with billing enabled
- The following APIs enabled:
  - BigQuery API
  - Vertex AI API
- Sufficient IAM permissions (BigQuery Admin, Vertex AI User)

## Step 1: Install & Import Dependencies

Install required Python packages. These are typically pre-installed in Colab Enterprise,
but we ensure the correct versions are available.

In [ ]:
# Install/upgrade necessary packages
!pip install --upgrade google-cloud-bigquery \
                        google-cloud-aiplatform \
                        db-dtypes \
                        pandas \
                        tqdm -q

In [ ]:
# ─────────────────────────────────────────────────────────────
# Core imports
# ─────────────────────────────────────────────────────────────
import time
import json
import textwrap

import pandas as pd
from tqdm.notebook import tqdm                    # Progress bars

# Google Cloud – BigQuery
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig, SourceFormat

# Vertex AI – Embeddings & Gemini
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel, Part

print("✅ All packages imported successfully.")

✅ All packages imported successfully.


## Step 2: Configuration

Set your Google Cloud **project ID** and preferred **region** below.
All resources (BigQuery dataset, Vertex AI calls) will use these settings.

In [ ]:
# ─────────────────────────────────────────────────────────────
# ⚙️  USER CONFIGURATION — update these values before running
# ─────────────────────────────────────────────────────────────
PROJECT_ID   = "qwiklabs-gcp-02-689e008843d4"       # e.g. "my-gcp-project-123"
REGION       = "us-central1"           # Vertex AI region
BQ_DATASET   = "aurora_bay"            # BigQuery dataset name (will be created)
BQ_TABLE     = "faqs"                  # Raw FAQ table
BQ_EMB_TABLE = "faqs_with_embeddings" # Table that stores embeddings

# Source data in Google Cloud Storage
GCS_URI = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Vertex AI model identifiers
EMBEDDING_MODEL = "text-embedding-005"  # Latest text embedding model
GEMINI_MODEL    = "gemini-2.0-flash-001" # Fast, capable Gemini model

# Number of similar FAQs to retrieve for each user question
TOP_K = 3

# ─────────────────────────────────────────────────────────────
# Initialise Vertex AI SDK
# ─────────────────────────────────────────────────────────────
vertexai.init(project=PROJECT_ID, location=REGION)

# Initialise BigQuery client
bq_client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Configuration complete.")
print(f"   Project : {PROJECT_ID}")
print(f"   Region  : {REGION}")
print(f"   Dataset : {BQ_DATASET}")

✅ Configuration complete.
   Project : qwiklabs-gcp-02-689e008843d4
   Region  : us-central1
   Dataset : aurora_bay


## Step 3: Create BigQuery Dataset & Import CSV

We create a BigQuery dataset (if it doesn't already exist) and then load the
Aurora Bay FAQ CSV directly from Google Cloud Storage.

In [ ]:
# ─────────────────────────────────────────────────────────────
# 3a. Create the BigQuery dataset (skip if already exists)
# ─────────────────────────────────────────────────────────────
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{BQ_DATASET}")
dataset_ref.location = "US"  # Multi-region for flexibility

try:
    bq_client.create_dataset(dataset_ref, exists_ok=True)
    print(f"✅ Dataset '{BQ_DATASET}' is ready.")
except Exception as e:
    print(f"⚠️  Could not create dataset: {e}")

✅ Dataset 'aurora_bay' is ready.


In [ ]:
# ─────────────────────────────────────────────────────────────
# 3b. Load the CSV from GCS into BigQuery
#
# The aurora-bay-faqs.csv has NO header row — the file starts
# directly with data. If we used autodetect with skip_leading_rows=1
# BigQuery would silently drop the first real data row and name
# columns 'string_field_0', 'string_field_1', etc.
#
# Fix: supply an explicit schema AND set skip_leading_rows=0
# so every row is treated as data with the names we define.
# WRITE_TRUNCATE replaces the table on each run (idempotent).
# ─────────────────────────────────────────────────────────────
table_ref = f"{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"

# Explicit schema: two STRING columns with meaningful names
faq_schema = [
    bigquery.SchemaField("question", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("answer",   "STRING", mode="NULLABLE"),
]

job_config = LoadJobConfig(
    source_format     = SourceFormat.CSV,
    skip_leading_rows = 0,              # No header row in this file
    schema            = faq_schema,     # Explicit column names & types
    write_disposition = "WRITE_TRUNCATE",
)

print(f"⏳ Loading data from:\n   {GCS_URI}")
print(f"   → {table_ref}\n")

load_job = bq_client.load_table_from_uri(
    GCS_URI, table_ref, job_config=job_config
)
load_job.result()  # Wait for the job to complete

# Retrieve and display table metadata
table = bq_client.get_table(table_ref)
print(f"✅ Load complete! Table has {table.num_rows:,} rows.")
print("\nSchema:")
for field in table.schema:
    print(f"  {field.name:30s} {field.field_type}")

⏳ Loading data from:
   gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv
   → qwiklabs-gcp-02-689e008843d4.aurora_bay.faqs

✅ Load complete! Table has 51 rows.

Schema:
  question                       STRING
  answer                         STRING


In [ ]:
# ─────────────────────────────────────────────────────────────
# 3c. Preview the imported data
# ─────────────────────────────────────────────────────────────
preview_query = f"""
    SELECT *
    FROM `{table_ref}`
    LIMIT 5
"""

df_preview = bq_client.query(preview_query).to_dataframe()
print(f"Preview of '{BQ_TABLE}' table:")
df_preview

Preview of 'faqs' table:


,question,answer
0,question,answer
1,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
2,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
3,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
4,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."


## Step 4: Generate Embeddings

We use Vertex AI's `text-embedding-005` model to create a semantic vector
for each FAQ entry. The embedding is computed over the **combined question
and answer text**, so that similarity search works on meaning, not just keywords.

> **Why combine question + answer?**  
> A user's query may match either the question wording *or* key terms in the
> answer. Embedding both together gives the richest representation.

In [ ]:
# ─────────────────────────────────────────────────────────────
# 4a. Load all FAQs from BigQuery into a Pandas DataFrame
# ─────────────────────────────────────────────────────────────
all_faqs_query = f"SELECT * FROM `{table_ref}`"
df_faqs = bq_client.query(all_faqs_query).to_dataframe()

print(f"Loaded {len(df_faqs):,} FAQ rows.")
print(f"Columns: {list(df_faqs.columns)}")

Loaded 51 FAQ rows.
Columns: ['question', 'answer']


In [ ]:
# ─────────────────────────────────────────────────────────────
# 4b. Set question and answer column names
#
# Because the CSV has no header row we supplied an explicit schema
# in Step 3b, so the column names are always 'question' and 'answer'.
# We confirm they exist and print a quick sanity check.
# ─────────────────────────────────────────────────────────────

QUESTION_COL = "question"
ANSWER_COL   = "answer"

# Verify the columns are present before proceeding
missing = [c for c in [QUESTION_COL, ANSWER_COL] if c not in df_faqs.columns]
if missing:
    raise ValueError(
        f"Expected columns not found: {missing}\n"
        f"Actual columns: {list(df_faqs.columns)}\n"
        "Re-run Step 3b to reload the table with the correct schema."
    )

print(f"Question column : '{QUESTION_COL}'")
print(f"Answer column   : '{ANSWER_COL}'")
print(f"\nSample values:")
print(df_faqs[[QUESTION_COL, ANSWER_COL]].head(3).to_string(index=False))

Question column : 'question'
Answer column   : 'answer'

Sample values:
                             question                                                                                                                                              answer
                             question                                                                                                                                              answer
         When was Aurora Bay founded?                                    Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.
What is the population of Aurora Bay? Aurora Bay has a population of approximately 3,200 residents, although it can fluctuate seasonally due to temporary fishing and tourism workforces.


In [ ]:
# ─────────────────────────────────────────────────────────────
# 4c. Helper function: generate embeddings via Vertex AI
#
# The API accepts up to 250 texts per batch. We process rows in
# batches of 100 to stay well within limits and avoid timeouts.
# ─────────────────────────────────────────────────────────────

embedding_model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL)

BATCH_SIZE = 100  # Texts per API call

def generate_embeddings_batch(texts: list[str]) -> list[list[float]]:
    """
    Generate text embeddings for a list of strings.

    Args:
        texts: List of text strings to embed.

    Returns:
        List of embedding vectors (one per input text).
    """
    all_embeddings = []

    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Embedding batches"):
        batch = texts[i : i + BATCH_SIZE]
        try:
            results = embedding_model.get_embeddings(batch)
            all_embeddings.extend([r.values for r in results])
        except Exception as e:
            print(f"⚠️  Error on batch {i//BATCH_SIZE}: {e}")
            # Fill with None so row indices remain aligned
            all_embeddings.extend([None] * len(batch))

        # Polite delay to avoid quota exhaustion on large datasets
        time.sleep(0.5)

    return all_embeddings

print(f"✅ Embedding model '{EMBEDDING_MODEL}' loaded.")

✅ Embedding model 'text-embedding-005' loaded.


/usr/local/lib/python3.12/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
# ─────────────────────────────────────────────────────────────
# 4d. Create combined text and generate embeddings
# ─────────────────────────────────────────────────────────────

# Build combined Q+A strings for embedding
df_faqs["combined_text"] = (
    "Question: " + df_faqs[QUESTION_COL].fillna("") +
    " Answer: "  + df_faqs[ANSWER_COL].fillna("")
)

print(f"Generating embeddings for {len(df_faqs)} FAQ entries...")
combined_texts = df_faqs["combined_text"].tolist()

embeddings = generate_embeddings_batch(combined_texts)

df_faqs["embedding"] = embeddings

# Verify – show dimension of first embedding
dim = len(df_faqs["embedding"].iloc[0])
print(f"\n✅ Embeddings generated! Dimension: {dim}")
print(f"   Rows with valid embeddings: {df_faqs['embedding'].notna().sum()}")

Generating embeddings for 51 FAQ entries...


Embedding batches:   0%|          | 0/1 [00:00<?, ?it/s]


✅ Embeddings generated! Dimension: 768
   Rows with valid embeddings: 51


## Step 5: Store Embeddings in BigQuery

We write the enriched DataFrame (original columns + embedding vectors) back to
a new BigQuery table. BigQuery natively supports `ARRAY<FLOAT64>` columns,
which is exactly what Vertex AI embeddings produce.

In [ ]:
# ─────────────────────────────────────────────────────────────
# 5a. Define explicit BigQuery schema for the embeddings table
#
# We keep all original columns and add:
#   combined_text – the text that was embedded
#   embedding     – the vector as ARRAY<FLOAT64>
# ─────────────────────────────────────────────────────────────

# Build schema dynamically from the DataFrame
type_map = {
    "object":   "STRING",
    "int64":    "INTEGER",
    "float64":  "FLOAT",
    "bool":     "BOOLEAN",
}

schema = []
for col in df_faqs.columns:
    if col == "embedding":
        # Embeddings are stored as ARRAY<FLOAT64>
        schema.append(
            bigquery.SchemaField(col, "FLOAT64", mode="REPEATED")
        )
    else:
        bq_type = type_map.get(str(df_faqs[col].dtype), "STRING")
        schema.append(
            bigquery.SchemaField(col, bq_type, mode="NULLABLE")
        )

print("BigQuery schema for embeddings table:")
for field in schema:
    print(f"  {field.name:30s} {field.field_type:12s} {field.mode}")

BigQuery schema for embeddings table:
  question                       STRING       NULLABLE
  answer                         STRING       NULLABLE
  combined_text                  STRING       NULLABLE
  embedding                      FLOAT64      REPEATED


In [ ]:
# ─────────────────────────────────────────────────────────────
# 5b. Write the enriched DataFrame to BigQuery
# ─────────────────────────────────────────────────────────────

emb_table_ref = f"{PROJECT_ID}.{BQ_DATASET}.{BQ_EMB_TABLE}"

job_config_write = bigquery.LoadJobConfig(
    schema            = schema,
    write_disposition = "WRITE_TRUNCATE",  # Overwrite on each run
)

print(f"⏳ Writing {len(df_faqs):,} rows to:\n   {emb_table_ref}")

write_job = bq_client.load_table_from_dataframe(
    df_faqs, emb_table_ref, job_config=job_config_write
)
write_job.result()  # Block until complete

written_table = bq_client.get_table(emb_table_ref)
print(f"\n✅ Embeddings table ready! Rows: {written_table.num_rows:,}")

⏳ Writing 51 rows to:
   qwiklabs-gcp-02-689e008843d4.aurora_bay.faqs_with_embeddings

✅ Embeddings table ready! Rows: 51


In [ ]:
# ─────────────────────────────────────────────────────────────
# 5c. Quick sanity check — peek at the stored embeddings
# ─────────────────────────────────────────────────────────────
sanity_query = f"""
    SELECT
        {QUESTION_COL},
        ARRAY_LENGTH(embedding)   AS embedding_dim,
        embedding[OFFSET(0)]      AS first_element
    FROM `{emb_table_ref}`
    LIMIT 5
"""

df_sanity = bq_client.query(sanity_query).to_dataframe()
print("Sample rows from the embeddings table:")
df_sanity

Sample rows from the embeddings table:


,question,embedding_dim,first_element
0,question,768,-0.068221
1,When was Aurora Bay founded?,768,-0.079507
2,What is the population of Aurora Bay?,768,-0.032302
3,Where is the Aurora Bay Town Hall located?,768,-0.068711
4,Who is the current mayor of Aurora Bay?,768,-0.075909


## Step 6: Build the Vector Search Function

We implement cosine similarity search **directly in BigQuery SQL**.


In [ ]:
# ─────────────────────────────────────────────────────────────
# 6a. Vector search helper
#
# This function:
#   1. Embeds the user's question using the same model
#   2. Runs a SQL query that computes cosine distance against
#      every stored embedding in BigQuery
#   3. Returns the top-K closest matches
# ─────────────────────────────────────────────────────────────

def vector_search(user_question: str, top_k: int = TOP_K) -> pd.DataFrame:
    """
    Find the most semantically similar FAQs to the user's question.

    Args:
        user_question: Natural language question from the user.
        top_k:         Number of top results to return.

    Returns:
        DataFrame with columns [question, answer, similarity_score]
        sorted by descending similarity.
    """
    # Step 1 – embed the user question
    q_embedding = embedding_model.get_embeddings([user_question])[0].values

    # Step 2 – format the vector as a BigQuery ARRAY literal
    array_literal = "[" + ", ".join(str(v) for v in q_embedding) + "]"

    # Step 3 – run cosine distance query in BigQuery
    # ML.DISTANCE returns cosine *distance* (0 = identical, 2 = opposite)
    # so we convert to similarity = 1 - distance for intuitive scoring.
    search_sql = f"""
        SELECT
            {QUESTION_COL}                            AS question,
            {ANSWER_COL}                              AS answer,
            ROUND(
                1 - ML.DISTANCE(
                        embedding,
                        {array_literal},
                        'COSINE'
                    ),
                4
            )                                         AS similarity_score
        FROM `{emb_table_ref}`
        ORDER BY similarity_score DESC
        LIMIT {top_k}
    """

    df_results = bq_client.query(search_sql).to_dataframe()
    return df_results


# ── Quick test ──────────────────────────────────────────────
test_question = "What are the visiting hours?"
print(f"Test query: '{test_question}'\n")
df_test = vector_search(test_question)
df_test

Test query: 'What are the visiting hours?'



,question,answer,similarity_score
0,Are there specific quiet hours or noise ordina...,Yes. Residential noise ordinances go into effe...,0.5379
1,What are the operating hours of the Aurora Bay...,The library is open Monday through Friday from...,0.5251
2,When are the town council meetings held?,Town council meetings are held every second Tu...,0.4952


## Step 7: Build the Gemini-Powered Chatbot

We combine the vector search results with the user's question into a
**retrieval-augmented generation (RAG)** prompt. Gemini uses the retrieved
FAQ context to produce a grounded, accurate answer.

In [ ]:
# ─────────────────────────────────────────────────────────────
# 7a. Load the Gemini model
# ─────────────────────────────────────────────────────────────

gemini = GenerativeModel(
    model_name = GEMINI_MODEL,
    system_instruction = (
        "You are a helpful and friendly customer-service assistant for Aurora Bay. "
        "Answer the user's question accurately using ONLY the FAQ context provided. "
        "If the context does not contain enough information to answer the question, "
        "say so politely and suggest the user contact Aurora Bay directly. "
        "Keep your answers concise and easy to understand."
    )
)

print(f"✅ Gemini model '{GEMINI_MODEL}' loaded.")

✅ Gemini model 'gemini-2.0-flash-001' loaded.


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
# ─────────────────────────────────────────────────────────────
# 7b. RAG prompt builder
#
# Formats the retrieved FAQ matches as readable context and
# appends the user's question for Gemini to answer.
# ─────────────────────────────────────────────────────────────

def build_rag_prompt(user_question: str, context_df: pd.DataFrame) -> str:
    """
    Construct a RAG prompt from the user question and retrieved FAQ context.

    Args:
        user_question: The question posed by the user.
        context_df:    DataFrame returned by vector_search().

    Returns:
        A formatted prompt string ready to send to Gemini.
    """
    context_blocks = []
    for i, row in context_df.iterrows():
        block = (
            f"--- FAQ {i+1} (similarity: {row['similarity_score']:.4f}) ---\n"
            f"Q: {row['question']}\n"
            f"A: {row['answer']}"
        )
        context_blocks.append(block)

    context_str = "\n\n".join(context_blocks)

    prompt = f"""\
Below are the most relevant Aurora Bay FAQ entries retrieved for this question.
Use them to answer the user's question accurately.

=== RETRIEVED FAQ CONTEXT ===
{context_str}

=== USER QUESTION ===
{user_question}

=== YOUR ANSWER ==="""

    return prompt

print("✅ RAG prompt builder defined.")

✅ RAG prompt builder defined.


In [ ]:
# ─────────────────────────────────────────────────────────────
# 7c. Main chatbot function
#
# Orchestrates: Vector Search → Prompt Building → Gemini → Answer
# ─────────────────────────────────────────────────────────────

def ask_aurora_bay(user_question: str, top_k: int = TOP_K, verbose: bool = True) -> str:
    """
    Answer a user question about Aurora Bay using RAG.

    Pipeline:
        1. Embed the question.
        2. Vector-search BigQuery for the top-K relevant FAQs.
        3. Build a RAG prompt with the retrieved context.
        4. Send the prompt to Gemini.
        5. Return Gemini's response.

    Args:
        user_question: The question to answer.
        top_k:         Number of FAQ results to retrieve.
        verbose:       If True, print retrieved FAQs before the answer.

    Returns:
        Gemini's answer as a string.
    """
    # ── Step 1 & 2: Vector Search ─────────────────────────────
    context_df = vector_search(user_question, top_k=top_k)

    if verbose:
        print("📋 Retrieved FAQ Context:")
        print("-" * 60)
        for _, row in context_df.iterrows():
            print(f"  [{row['similarity_score']:.4f}] {row['question']}")
        print("-" * 60)

    # ── Step 3: Build RAG Prompt ──────────────────────────────
    prompt = build_rag_prompt(user_question, context_df)

    # ── Step 4: Send to Gemini ────────────────────────────────
    response = gemini.generate_content(prompt)

    # ── Step 5: Extract and return the answer ─────────────────
    answer = response.text.strip()
    return answer

print("✅ Chatbot function defined.")

✅ Chatbot function defined.


## Step 8: Test the Chatbot

Run the chatbot against a few sample questions to verify end-to-end functionality.

In [ ]:
# ─────────────────────────────────────────────────────────────
# Test 1 – Visiting hours
# ─────────────────────────────────────────────────────────────
question_1 = "What are the visiting hours at Aurora Bay?"

print(f"\n{'='*65}")
print(f" 🧑 User: {question_1}")
print(f"{'='*65}\n")

answer_1 = ask_aurora_bay(question_1)

print(f"\n🤖 Aurora Bay Assistant:")
print(textwrap.fill(answer_1, width=70))


 🧑 User: What are the visiting hours at Aurora Bay?

📋 Retrieved FAQ Context:
------------------------------------------------------------
  [0.7392] What are the operating hours of the Aurora Bay Public Library?
  [0.6962] Is there a local hospital in Aurora Bay?
  [0.6669] Does Aurora Bay have public transportation?
------------------------------------------------------------

🤖 Aurora Bay Assistant:
I am sorry, I do not have the information about visiting hours. Please
contact Aurora Bay directly for more information.


In [ ]:
# ─────────────────────────────────────────────────────────────
# Test 2 – Parking / Getting there
# ─────────────────────────────────────────────────────────────
question_2 = "Is there parking available?"

print(f"\n{'='*65}")
print(f" 🧑 User: {question_2}")
print(f"{'='*65}\n")

answer_2 = ask_aurora_bay(question_2)

print(f"\n🤖 Aurora Bay Assistant:")
print(textwrap.fill(answer_2, width=70))


 🧑 User: Is there parking available?

📋 Retrieved FAQ Context:
------------------------------------------------------------
  [0.5288] What are the winter road conditions usually like?
  [0.5006] Does Aurora Bay have public transportation?
  [0.4874] Are there specific quiet hours or noise ordinances?
------------------------------------------------------------

🤖 Aurora Bay Assistant:
I am sorry, but the FAQ does not contain information about parking.
Please contact Aurora Bay directly for more information.


In [ ]:
# ─────────────────────────────────────────────────────────────
# Test 3 – Out-of-scope question (graceful degradation)
# ─────────────────────────────────────────────────────────────
question_3 = "Can I bring my pet dog?"

print(f"\n{'='*65}")
print(f" 🧑 User: {question_3}")
print(f"{'='*65}\n")

answer_3 = ask_aurora_bay(question_3)

print(f"\n🤖 Aurora Bay Assistant:")
print(textwrap.fill(answer_3, width=70))


 🧑 User: Can I bring my pet dog?

📋 Retrieved FAQ Context:
------------------------------------------------------------
  [0.5025] How do I adopt a pet from the local shelter?
  [0.4675] Are there specific quiet hours or noise ordinances?
  [0.4526] What is the procedure for hosting an event at the waterfront park?
------------------------------------------------------------

🤖 Aurora Bay Assistant:
This question cannot be answered from the given context. Please
contact Aurora Bay directly for more information.


## Summary

| Step | What was done |
|------|---------------|
| **1** | Installed and imported all required libraries |
| **2** | Configured GCP project, dataset, and model settings |
| **3** | Created a BigQuery dataset and loaded the FAQ CSV from GCS |
| **4** | Generated semantic embeddings for each Q&A pair using `text-embedding-005` |
| **5** | Stored the enriched data (including `ARRAY<FLOAT64>` embeddings) back to BigQuery |
| **6** | Built a vector search function using BigQuery's `ML.DISTANCE` (cosine similarity) |
| **7** | Built a RAG-based chatbot that passes retrieved context to Gemini |
| **8** | Tested the chatbot with sample questions |
| **9** | Provided an interactive chat loop for live use |

### Key Design Decisions

- **Embedding Q+A together** maximises the richness of each vector and improves retrieval recall.
- **BigQuery for vector search** keeps everything in one managed service — no separate vector database needed.
- **RAG over fine-tuning** is cheaper, faster to update (just reload the CSV), and more transparent.
- **Gemini system instruction** anchors the model to FAQ-only responses, preventing hallucination.

### Next Steps

- **Scheduled re-embedding**: Use Cloud Scheduler + Cloud Run to refresh embeddings when the FAQ CSV is updated.
- **Conversation history**: Pass previous turns to Gemini for multi-turn dialogue.
- **Evaluation**: Log user questions and thumbs-up/down feedback for continuous improvement.
- **Scaling**: For very large FAQ sets (>1 M rows), consider migrating the vector index to Vertex AI Vector Search (Matching Engine) for sub-millisecond ANN retrieval.